<a href="https://colab.research.google.com/github/akrm3008/deep-gasoline/blob/master/RAG_on_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.7 MB/s eta 0:00:00


In [ ]:
!pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 28.7 MB/s eta 0:00:00


In [ ]:
## building simple RAG using huggingface models
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import datasets
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/tweet_text_2.csv", encoding = 'cp1252')
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
model = AutoModel.from_pretrained(EMBEDDING_MODEL_NAME)
docs = list(data["TWEET_TEXT"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

In [ ]:
data['TWEET_TEXT']

,TWEET_TEXT
0,Can someone explain to me why everyone is fill...
1,The drive and wait I had to make for gas is cr...
2,In the trunk of your car you need two gas cans...
3,If the hurricane hits and everyone tries to le...
4,I need to stop driving like I have a race car ...
...,...
4065,These gas prices are fucking OD
4066,Who tryna gas tho ???????
4067,"It'll be easy to start the generator, they sai..."
4068,Bitch i do this so the gas doesn't get all ove...


In [ ]:
doc_lens = [len(doc) for doc in docs]
max(doc_lens)

161

In [ ]:
chunk_size = 512
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True
    )
docs_processed = []
for doc in docs:
  docs_processed += text_splitter.create_documents([doc])


In [ ]:
docs_processed[2]

Document(metadata={'start_index': 0}, page_content='In the trunk of your car you need two gas cans case of water first aid kit and a road kit at least. Just in case of emergency')

In [ ]:
docs_processed

[Document(metadata={'start_index': 0}, page_content='Can someone explain to me why everyone is filling up their gas tanks?'),
 Document(metadata={'start_index': 0}, page_content='The drive and wait I had to make for gas is crazy af'),
 Document(metadata={'start_index': 0}, page_content='In the trunk of your car you need two gas cans case of water first aid kit and a road kit at least. Just in case of emergency'),
 Document(metadata={'start_index': 0}, page_content='If the hurricane hits and everyone tries to leave it will be impossible for everyone to leave. Yet we all have gas'),
 Document(metadata={'start_index': 0}, page_content='I need to stop driving like I have a race car since gas stations are running out of gas'),
 Document(metadata={'start_index': 0}, page_content="I need gas so bad and I'm at work . Hopefully I can find some after ??"),
 Document(metadata={'start_index': 0}, page_content='Bless the gas stations that are trying to organize traffic flow'),
 Document(metadata={'

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

<ipython-input-8-b0c7b39a7f62>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
user_query = "What are the issues people facing in Florida due to Hurricane?"

In [ ]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=1000)

In [ ]:
retrieved_docs

[Document(metadata={'start_index': 0}, page_content='Living in Florida right now is like being in an episode of the Walking Dead... limited food, gas, supplies... and everyones acting crazy'),
 Document(metadata={'start_index': 0}, page_content='I hate the people saying that everyone in Florida needs to evacuate. There is not enough gas and too much traffic for everyone to get out'),
 Document(metadata={'start_index': 0}, page_content="This is so stressful it's unreal. There's no gas for miles, no hotels available for hundreds. I'm scared to be living in FL right now"),
 Document(metadata={'start_index': 0}, page_content='I dead ass hate Miami. People actually need gas for work y los comepingas estos taking up all the gas bc of a fucking hurricane.'),
 Document(metadata={'start_index': 0}, page_content="FUCK this hurricane and the effect it has had on gas prices, traffic, and lines in public places. I'm over it."),
 Document(metadata={'start_index': 0}, page_content="Trapped in the hur

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]

In [ ]:
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

In [ ]:
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]

In [ ]:
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
final_prompt = RAG_PROMPT_TEMPLATE.format(question="What are the issues people facing in Florida due to Hurricane?", context=context)

In [ ]:
answer = READER_LLM(final_prompt)[0]["generated_text"]

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (32768). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
print(answer)

People in Florida are facing various issues due to Hurricane Irma, including gas shortages, water shortages, power outages, and property damage. Many gas stations are running low on fuel or have completely run out, making it difficult for people to fill up their tanks. Some supermarkets and gas stations are closed, while others are experiencing long lines and high prices due to gas price gouging. Additionally, some traffic signals are malfunctioning, causing traffic congestion in certain areas. Property damage, including fallen trees and debris blocking roads, is also a significant issue in some regions. People are advised to evacuate if possible, but some are struggling to find transportation due to gridlock on major highways. Overall, the hurricane has caused widespread disruption and chaos in Florida.


In [ ]:
query2 = "Where are people facing gas shortage in Florida?"
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=query2, k=1000)
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
final_prompt = RAG_PROMPT_TEMPLATE.format(question=query2, context=context)
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)


Based on the social media posts compiled above, it appears that people facing gas shortage in Florida include those in areas affected by Hurricane Irma, such as Miami, Tampa, St. Petersburg, Naples, Marco Island, Homestead, and Lake Worth. Some specific locations mentioned include Wawa on 5th Ave N and Biscayne Ranch, Shell gas station on 36th Street and Sunset, RaceTrac in Brandon Hills, FL, Walmart on Fleming Island, Shell on Hallandale Beach Blvd and NW 8th St, Chevron at 62nd Avenue and Sunset in South Miami, Walmart on 177th St and 2nd Ave, and the Shell on 27th & US1 in South Miami. It should be noted that some gas stations are only selling unleaded gas or premium gas, and lines for gas are long and moving slowly due to heavy traffic. Additionally, some locations are reporting that gas is running low or out, and people are waiting in line for hours to fill up their tanks. The exact extent and severity of the gas shortage is unclear, but it seems to be affecting a significant numb

In [ ]:
query3 = "Can you list all the gas stations out of gas and their locations?"
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=query3, k=1000)
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
final_prompt = RAG_PROMPT_TEMPLATE.format(question=query3, context=context)
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

I do not have access to real-time information or the ability to physically locate gas stations. However, based on the text provided, it mentions various gas stations throughout florida that have run out of gas or have long lines due to the impact of hurricane irma. Some specific locations mentioned include:

- wawa on 14th st in bradenton, fl
- citgo on hallandale beach blvd &amp; nw 27 ave 41 st, miami, fl 33018
- hillsborough county, fl 33612
- altamonte springs station 32714
- mobil on miramar pkwy by everglades
- jensen beach (thanks to @rockafowlers)
- wawa on 14th st in bradenton, fl
- wawa after seeing the gas lines
- pilot off 75, exit 358, hwy 326, in ocala, fl
- wawa off a1a and granada for lunch, traffic is backed up for sandbags and propane!!!
- shell on hallandale beach blvd &amp; nw 8th st, miami, fl 33018
- princeton walk tc in tucker, ga
- wawa in east naples
- wawa in lake worth
- wawa in sarasota
- wawa on i-75, exit 358, hwy 326, in ocala, fl
- wawa on nw 27th ave, h

In [ ]:
from transformers import Pipeline
from ragatouille import RAGPretrainedModel
from typing import List, Optional, Tuple
from langchain.docstore.document import Document as LangchainDocument

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")


def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
